# Galilean Satellites - Initial State Estimation
Copyright (c) 2010-2022, Delft University of Technology. All rights reserved. This file is part of the Tudat. Redistribution and use in source and binary forms, with or without modification, are permitted exclusively under the terms of the Modified BSD license. You should have received a copy of the license with this file. If not, please or visit: http://tudat.tudelft.nl/LICENSE.

## Context

## Import Statements
Typically - in the most pythonic way - all required modules are imported at the very beginning.

Some standard modules are first loaded: `numpy` and `matplotlib.pyplot`. Within this example, while no particular new functionality of `tudatpy` will be introduced, we will nevertheless explore the already known parts of the `estimation` module in more depth and how it can be applied to intricate problems.

In [6]:
# General imports
import os
import math
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

# tudatpy imports
from tudatpy.kernel import constants
from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation
from tudatpy.kernel.astro import time_conversion, element_conversion
from tudatpy.kernel.numerical_simulation import environment_setup
from tudatpy.kernel.numerical_simulation import propagation_setup
from tudatpy.kernel.numerical_simulation import estimation, estimation_setup

# Problem-specific imports
import galilean_moons_state_estimation_util as Util

In [14]:
for filename in os.listdir(os.path.join(os.getcwd(), 'TLEs')):
    with open(os.path.join(os.getcwd(), 'TLEs', filename), 'r') as f:

        lines = f.read().splitlines()

        #Calculate epoch of TLE
        tle_year = float(lines[0][18:20])
        tle_day_of_year = float(lines[0][20:32])
        if tle_year >= 60.0:
            epoch_year = 1900 + tle_year
        else:
            epoch_year = 2000 + tle_year

        tle_epoch = int(epoch_year * constants.JULIAN_YEAR + tle_day_of_year * constants.JULIAN_DAY)

        foo = spice.get_cartesian_state_from_tle_at_epoch(tle_epoch, f)
        print(foo)

TypeError: get_cartesian_state_from_tle_at_epoch(): incompatible function arguments. The following argument types are supported:
    1. (epoch: float, tle: tudat::ephemerides::Tle) -> numpy.ndarray[numpy.float64[6, 1]]

Invoked with: 63855112523, <_io.TextIOWrapper name='/Users/kai/Documents/tu_delft/tudat/tudatpy-examples/estimation/TLEs/25544_epoch_63855112523.dat' mode='r' encoding='UTF-8'>

In [ ]:
# Create default body settings for selected celestial bodies
bodies_to_create = ['Sun', 'Earth', 'Moon', 'Mars', 'Venus', 'Jupiter']

# Create default body settings for bodies_to_create
global_frame_origin = 'Earth'
global_frame_orientation = 'ECLIPJ2000'
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation)

### EARTH ENVIRONMENT MODELS ###
# Atmosphere
body_settings.get('Earth').atmosphere_settings = environment_setup.atmosphere.nrlmsise00()
# Rotation
body_settings.get('Earth').rotation_model_settings = environment_setup.rotation_model.gcrs_to_itrs(
    precession_nutation_theory=environment_setup.rotation_model.IAUConventions.iau_2006, base_frame='ECLIPJ2000')
# Gravity-field reference frame
body_settings.get('Earth').gravity_field_settings.associated_reference_frame = 'ITRS'
# Shape
body_settings.get('Earth').shape_settings = environment_setup.shape.oblate_spherical(6378137.0, 1.0 / 298.257223563)

# Create system of selected bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

### ACCELERATION SETTINGS ###
# Define bodies that are propagated, and their central bodies of propagation
bodies_to_propagate = ['sc_name']
central_bodies = ['Earth']

accelerations_settings_on_sc = dict(
    Sun=[
        propagation_setup.acceleration.cannonball_radiation_pressure(),
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Earth=[
        propagation_setup.acceleration.spherical_harmonic_gravity(32, 32),
        propagation_setup.acceleration.aerodynamic()
    ],
    Moon=[
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Mars=[
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Venus=[
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Jupiter=[
        propagation_setup.acceleration.point_mass_gravity()
    ]
)

# Create acceleration models
acceleration_models = propagation_setup.create_acceleration_models(
    bodies, acceleration_settings_on_sc, bodies_to_propagate, central_bodies)

# Define initial state
initial_states = list()
for body in bodies_to_propagate:
    initial_states.append(spice.get_body_cartesian_state_at_epoch(
        target_body_name=body,
        observer_body_name='Jupiter',
        reference_frame_name='ECLIPJ2000',
        aberration_corrections='none',
        ephemeris_time=simulation_start_epoch))
initial_states = np.concatenate(initial_states)

### INTEGRATOR SETTINGS ###
# Use fixed step-size integrator (RK4) with fixed time-step of 10 seconds
# Create integrator settings
time_step_sec = 10.0
integrator_settings = propagation_setup.integrator. \
    runge_kutta_fixed_step_size(initial_time_step=time_step_sec,
                                coefficient_set=propagation_setup.integrator.CoefficientSets.rk4)

### TERMINATION SETTINGS ###
termination_condition = propagation_setup.propagator.time_termination(simulation_end_epoch)

### PROPAGATOR SETTINGS ###
propagator_settings = propagation_setup.propagator. \
    translational(central_bodies=central_bodies,
                  acceleration_models=acceleration_models,
                  bodies_to_integrate=bodies_to_propagate,
                  initial_states=initial_states,
                  initial_time=simulation_start_epoch,
                  integrator_settings=integrator_settings,
                  termination_settings=termination_condition,
                  output_variables=dependent_variables_to_save)

In [ ]:




link_ends_io = dict()
link_ends_io[estimation_setup.observation.observed_body] = estimation_setup.observation.\
    body_origin_link_end_id('Io')
link_definition_io = estimation_setup.observation.LinkDefinition(link_ends_io)

position_observation_settings = [estimation_setup.observation.cartesian_position(link_definition_io)]

parameters_to_estimate_settings = estimation_setup.parameter.initial_states(propagator_settings, bodies)
parameters_to_estimate = estimation_setup.create_parameter_set(parameters_to_estimate_settings, bodies)
original_parameter_vector = parameters_to_estimate.parameter_vector

print('Running propagation...')
estimator = numerical_simulation.Estimator(bodies, parameters_to_estimate,
                                           position_observation_settings, propagator_settings)